In [1]:
import pandas as pd
import os

In [2]:
col_desc=pd.read_csv("datadesc.csv")
col_desc.head()

,Field,Columns,Length,Description
0,1,1,1,Record Type
1,2,2-3,2,FIPS State Code
2,3,4-5,2,Functional Classification
3,4,6-11,6,Station Identification
4,5,12,1,Direction of Travel


In [26]:
statemap=pd.read_excel("State Mapping.xlsx",header=0)
statemap.columns

Index(['State', 'Code', 'Country'], dtype='object')

In [6]:
def somefunc(x):
    edict={}
    if not x.isnumeric():
        x=x.strip()
        rows=x.split("-")
        edict["Columns_start"]=int(rows[0])
        edict["Columns_end"]=int(rows[1])
    else:
        edict["Columns_start"]=int(x)
        edict["Columns_end"]=int(x)
    return pd.Series(edict)

In [7]:
col_desc[["Columns_start","Columns_end"]]=col_desc["Columns"].apply(somefunc)
col_desc.drop(["Columns","Field"],inplace=True,axis=1)
col_desc["Length"]=col_desc["Length"].cumsum()
col_desc.head()

,Length,Description,Columns_start,Columns_end
0,1,Record Type,1,1
1,3,FIPS State Code,2,3
2,5,Functional Classification,4,5
3,11,Station Identification,6,11
4,12,Direction of Travel,12,12


In [8]:
col_desc["start_idx"]=col_desc["Length"].shift(1).fillna(0).astype(int)
col_desc=col_desc.rename(columns={"Length": "end_idx"})

In [9]:
col_desc.head()

,end_idx,Description,Columns_start,Columns_end,start_idx
0,1,Record Type,1,1,0
1,3,FIPS State Code,2,3,1
2,5,Functional Classification,4,5,3
3,11,Station Identification,6,11,5
4,12,Direction of Travel,12,12,11


In [10]:
def formatting(x):
     x=str(x)
     coldict={}
     for i,v in enumerate(col_desc["Description"]):
        coldict[str(v)]=x[col_desc["start_idx"][i]:col_desc["end_idx"][i]]
     return pd.Series(coldict)
data=pd.read_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Data\2011\\1. JAN\\AK_JAN_2011 (TMAS).VOL",delimiter="|",header=None)
# data[col_desc["Description"]]=data.iloc[:,0].apply(formatting)
# data.drop(columns=data.columns[0],axis=1,inplace=True)


In [19]:
# yearlist=os.listdir(r"C:\Users\ch14b\Desktop\DVA\Project\Data")
# yearlist=yearlist[4:-4]
#
# for y in yearlist:
y="2020"
cleaned_data=[]
monthlist=os.listdir(r"C:\Users\ch14b\Desktop\DVA\Project\Data"+"\\"+y)
monthlist=[i for i in monthlist if "station" not in i.lower()]
for m in monthlist:
    path=r"C:\Users\ch14b\Desktop\DVA\Project\Data"+"\\"+y+"\\"+m
    filelist=os.listdir(path)
    for f in filelist:
        try:
            data=pd.read_csv(path+"\\"+f)
            data[col_desc["Description"]]=data.iloc[:,0].apply(formatting)
            data.drop(columns=data.columns[0],axis=1,inplace=True)
            data[data.columns]=data[data.columns].apply(pd.to_numeric,downcast="integer",errors="ignore")
            data=data.fillna(data.mean())
            Traffic_Volume=pd.Series([i for i in data.columns if "hour" in i])
            data["Daily_Traffic"]=data[Traffic_Volume].sum(axis=1)
            data.drop(columns=Traffic_Volume,inplace=True)
            cleaned_data.append(data)
        except:
            print(path)
            continue


C:\Users\ch14b\AppData\Local\Temp\ipykernel_15300\916606694.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data=data.fillna(data.mean())
C:\Users\ch14b\AppData\Local\Temp\ipykernel_15300\916606694.py:19: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  Traffic_Volume=pd.Series([i for i in data.columns if "hour" in i])
C:\Users\ch14b\AppData\Local\Temp\ipykernel_15300\916606694.py:18: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  data=data.fillna(data.mean())
C:\Users\ch14b\AppData\Local\Temp\ipykernel_15300\916606694.py:19: FutureWarn

In [25]:
cleaned_data[0].columns

Index(['Record Type', 'FIPS State Code', 'Functional Classification',
       'Station Identification', 'Direction of Travel', 'Lane of Travel',
       'Year of Data', 'Month of Data', 'Day of Data', 'Day of Week',
       'Traffic Volume Counted, 00:01 - 01:00',
       'Traffic Volume Counted, 01:01 - 02:00',
       'Traffic Volume Counted, 02:01 - 03:00',
       'Traffic Volume Counted, 03:01 - 04:00',
       'Traffic Volume Counted, 04:01 - 05:00',
       'Traffic Volume Counted, 05:01 - 06:00',
       'Traffic Volume Counted, 06:01 - 07:00',
       'Traffic Volume Counted, 07:01 - 08:00',
       'Traffic Volume Counted, 08:01 - 09:00',
       'Traffic Volume Counted, 09:01 - 10:00',
       'Traffic Volume Counted, 10:01 - 11:00',
       'Traffic Volume Counted, 11:01 - 12:00',
       'Traffic Volume Counted, 12:01 - 13:00',
       'Traffic Volume Counted, 13:01 - 14:00',
       'Traffic Volume Counted, 14:01 - 15:00',
       'Traffic Volume Counted, 15:01 - 16:00',
       'Traffic Vo

In [30]:
for i in range(len(cleaned_data)):
    cleaned_data[i]=pd.merge(cleaned_data[i],statemap,left_on="FIPS State Code",right_on="Code")

In [31]:
cleaned_data[0].head()

,Record Type,FIPS State Code,Functional Classification,Station Identification,Direction of Travel,Lane of Travel,Year of Data,Month of Data,Day of Data,Day of Week,...,"Traffic Volume Counted, 19:01 - 20:00","Traffic Volume Counted, 20:01 - 21:00","Traffic Volume Counted, 21:01 - 22:00","Traffic Volume Counted, 22:01 - 23:00","Traffic Volume Counted, 23:01 - 24:00",Restrictions,Daily_Traffic,State,Code,Country
0,3,2,1R,101,1,1,20,1,3,6,...,9,6,13,7,4,0,0.0,Alaska,2,USA
1,3,2,1R,101,1,1,20,1,4,7,...,11,7,6,5,2,0,0.0,Alaska,2,USA
2,3,2,1R,101,1,1,20,1,5,1,...,20,14,4,3,7,0,0.0,Alaska,2,USA
3,3,2,1R,101,1,1,20,1,6,2,...,10,10,12,2,9,0,0.0,Alaska,2,USA
4,3,2,1R,101,1,1,20,1,7,3,...,4,7,11,7,5,0,0.0,Alaska,2,USA


In [38]:
pd.concat(cleaned_data).to_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Cleaned_data\2020.csv")

In [ ]:
# debug=pd.read_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Data\2021\NV_SEP_2021 (TMAS).VOL",delimiter="|")
# debug[debug.columns]=debug[debug.columns].apply(pd.to_numeric,downcast="integer",errors="ignore")
# debug=debug.fillna(debug.mean())
# Traffic_Volume=pd.Series([i for i in debug.columns if "hour" in i])
# debug["Daily_Traffic"]=debug[Traffic_Volume].sum(axis=1)
# debug.drop(columns=Traffic_Volume,inplace=True)
# # debug.head()
# debug=pd.read_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Cleaned_data\2021_3.csv")
# debug.head()

In [86]:
# year_output=pd.concat(cleaned_data,ignore_index=True)
# year_output.to_csv(path_or_buf=r"C:\Users\ch14b\Desktop\DVA\Project\Cleaned_data\\"+y+"_3.csv")

In [32]:
sdatadesc=pd.read_csv("Stationdata_desc.csv")
sdatadesc.drop(columns=sdatadesc.columns[-1],inplace=True)
sdatadesc[["Columns_start","Columns_end"]]=sdatadesc["Columns "].apply(somefunc)
sdatadesc.drop(["Columns ","Field "],inplace=True,axis=1)
sdatadesc["Length"]=sdatadesc["Width "].cumsum()

sdatadesc["start_idx"]=sdatadesc["Length"].shift(1).fillna(0).astype(int)
sdatadesc=sdatadesc.rename(columns={"Length": "end_idx"})
sdatadesc.drop(["Width "],inplace=True,axis=1)
sdatadesc.head()
# print(x)

,Description,Columns_start,Columns_end,end_idx,start_idx
0,Record Type,1,1,1,0
1,FIPS State Code,2,3,3,1
2,Station ID,4,9,9,3
3,Direction of Travel Code,10,10,10,9
4,Lane of Travel,11,11,11,10


In [35]:
def formatting2(x):
    x=str(x)
    coldict={}
    for i,v in enumerate(sdatadesc["Description "]):
        coldict[str(v)]=x[sdatadesc["start_idx"][i]:sdatadesc["end_idx"][i]]
    return pd.Series(coldict)

In [34]:
sdata=pd.read_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\TMAS20.sta",delimiter="|",header=None)
sdata[sdatadesc["Description "]]=sdata.iloc[:,0].apply(formatting2)


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\ch14b\\Desktop\\DVA\\Project\\Data\\StationData\\TMAS20.sta'

In [36]:
sdata.head()

NameError: name 'sdata' is not defined

In [8]:
sdata.drop(columns=sdata.columns[0],inplace=True)

In [9]:
sdata[sdata.columns]=sdata[sdata.columns].apply(pd.to_numeric,downcast="integer",errors="ignore")

In [10]:
sdata.columns

Index(['Record Type ', 'FIPS State Code ', 'Station ID ',
       'Direction of Travel Code ', 'Lane of Travel ', 'Year of Data ',
       'Functional Classification Code ', 'Number of Lanes in Direction',
       'Sample Type for Traffic Volume ',
       'Number of Lanes Monitored for Traffic Volume ',
       'Method of Traffic Volume Counting ',
       'Sample Type for Vehicle Classification ',
       'Number of Lanes Monitored for Vehicle Class. ',
       'Method of Vehicle Classification ',
       'Algorithm for Vehicle Classification ',
       'Classification System for Vehicle Classification ',
       'Sample Type for Truck Weight ',
       'Number of Lanes Monitored for Truck Weight ',
       'Method of Truck Weighing ', 'Calibration of Weighing System ',
       'Method of Data Retrieval ', 'Type of Sensor ',
       'Second Type of Sensor ', 'Primary Purpose - NEW ',
       'LRS Identification - NEW ', 'LRS Location Point - NEW ',
       'Latitude - NEW ', 'Longitude - NEW ',
     

In [25]:
# yearlist=os.listdir(r"C:\Users\ch14b\Desktop\DVA\Project\Data")
# yearlist=yearlist[1:-7]
# for y in yearlist:
y="2020"
print(y)
print(r"C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\\"+y)
filelist=os.listdir(r"C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\\"+y)
merged=[]
for f in filelist:
    # sdata=pd.read_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\TMAS2021.txt",delimiter="|")
    path=r"C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\"+f
    print(path)
    sdata=pd.read_csv(path,encoding="ISO-8859-1",on_bad_lines="skip",header=None)
    sdata[sdatadesc["Description "]]=sdata.iloc[:,0].apply(formatting2)
    sdata=sdata[["Station ID ","Direction of Travel Code ","Lane of Travel ","Latitude - NEW ","Longitude - NEW ","FIPS State Code"]]
    sdata[sdata.columns]=sdata[sdata.columns].apply(pd.to_numeric,downcast="integer",errors="coerce")
    # sdata=sdata[sdata["Latitude"]!=0
    merged.append(sdata)

stationdata=pd.concat(merged,ignore_index=True)
stationdata.to_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020.csv")
sdata.head()


2020
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\\2020
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\AK_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\AL_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\AR_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\AZ_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\CA_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\CO_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\CT_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\DC_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\DE_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\FL_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\GA_2020 (TMAS).STA
C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\\HI_2020 (TMAS).STA
C:\Users\ch14

,Station ID,Direction of Travel Code,Lane of Travel,Latitude - NEW,Longitude - NEW
0,1.0,3,1,41113250,104931200
1,1.0,3,2,41113250,104931200
2,1.0,7,1,41113250,104931200
3,1.0,7,2,41113250,104931200
4,2.0,1,1,42834750,105977870


In [26]:
stationdata.dtypes

Station ID                   float64
Direction of Travel Code        int8
Lane of Travel                  int8
Latitude - NEW                 int32
Longitude - NEW                int32
dtype: object

In [21]:
trial=pd.read_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020\MA_2020 (TMAS).STA",)
trial.head()

,S2500000111203U1T13H13F14T15Z1LPP0000000SR8NB 42485615 73202094 18600000N 2.36053E+11N3 80 route 8 S.OF LANESBOROUGH T.L.
0,S2500000151203U1T13H13F14T15Z1LPP0000000SR8NB ...
1,S2500000231204U3T33H33F13T35Z1LPP000000US20EB ...
2,S2500000232204U3T33H33F13T35Z1LPP000000US20EB ...
3,S2500000233204U3T33H33F13T35Z1LPP000000US20EB ...
4,S2500000271204U1T13H13F13T15Z1LPP000000US20EB ...


In [12]:
print(sdata.shape)


(36522, 5)


In [39]:
y="2020"

In [43]:
data=pd.read_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Cleaned_data\2020.csv")

C:\Users\ch14b\AppData\Local\Temp\ipykernel_15300\3043526001.py:1: DtypeWarning: Columns (4,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Cleaned_data\2020.csv")


In [44]:
sdata=pd.read_csv(r"C:\Users\ch14b\Desktop\DVA\Project\Data\StationData\2020stationdata.csv")

In [49]:
data.head()
data.drop(columns=data.columns[0],inplace=True)
data[data.columns]=data[data.columns].apply(pd.to_numeric,downcast="integer",errors="ignore")

,Unnamed: 0,Record Type,FIPS State Code,Functional Classification,Station Identification,Direction of Travel,Lane of Travel,Year of Data,Month of Data,Day of Data,...,"Traffic Volume Counted, 19:01 - 20:00","Traffic Volume Counted, 20:01 - 21:00","Traffic Volume Counted, 21:01 - 22:00","Traffic Volume Counted, 22:01 - 23:00","Traffic Volume Counted, 23:01 - 24:00",Restrictions,Daily_Traffic,State,Code,Country
0,0,3,2,1R,101,1,1,20,1,3,...,9,6,13,7,4,0,0.0,Alaska,2,USA
1,1,3,2,1R,101,1,1,20,1,4,...,11,7,6,5,2,0,0.0,Alaska,2,USA
2,2,3,2,1R,101,1,1,20,1,5,...,20,14,4,3,7,0,0.0,Alaska,2,USA
3,3,3,2,1R,101,1,1,20,1,6,...,10,10,12,2,9,0,0.0,Alaska,2,USA
4,4,3,2,1R,101,1,1,20,1,7,...,4,7,11,7,5,0,0.0,Alaska,2,USA


In [47]:
sdata

(31472, 6)

In [45]:
data=pd.merge(data,sdata,left_on=["station_id","travel_dir","travel_lane"],right_on=["Station_Id","Travel_Dir","Travel_Lane"])

KeyError: 'Station_Id'

In [ ]:
data.shape

In [ ]:
data.to_csv(path_or_buf=r"C:\Users\ch14b\Desktop\DVA\Project\Cleaned_merged\\"+y+".csv")